In [1]:
!pip install -qU pandas==2.2.3 pinecone==6.0.2


API KEY : pcsk_PzpSB_4DXSenzc29j2oe8YK5f3az3RGXhKKsQTXGmhzNfv6RqDWHTnaoVF9ihystmTH6p

In [19]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_PzpSB_4DXSenzc29j2oe8YK5f3az3RGXhKKsQTXGmhzNfv6RqDWHTnaoVF9ihystmTH6p")

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


def _initialize_text_splitter() -> RecursiveCharacterTextSplitter:
    return RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=1000,
        chunk_overlap=200,
    )


text_splitter = _initialize_text_splitter()

# Read The Document

In [3]:

from pypdf import PdfReader

documents = PdfReader("./documents/JUKNIS SPMB JATIM 2025_sign/JUKNIS SPMB JATIM 2025_sign.pdf")

# filter the empty string
pdf_texts = [p.extract_text().strip() for p in documents.pages]
pdf_texts = [text for text in pdf_texts if text]

print(pdf_texts)



['Juknis SPMB Jatim Tahun Ajaran 2025/2026  i \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nNomor : 100.3.6/1425/101.7.1/2025', 'Juknis SPMB Jatim Tahun Ajaran 2025/2026  ii \n \n \n \n \n \n \n \n \nKepala Dinas Pendidikan Provinsi Jawa Timur \n \nSesuai dengan Undang-Undang Dasar Negara Republik Indonesia Tahun \n1945 bahwa setiap warga negara berhak mendapat pendidikan yang \nbermutu dan berkeadilan. Peningkatan dan pemerataan mutu pendidikan \nmenjadi tantangan utama dalam pembangunan pendidikan di Jawa Timur. \nPemerintah Jawa Timur telah melakukan banyak hal untuk memperbaiki \ndan meningkatkan kualitas pendidikan yang bermutu dan berkeadilan, \nsalah satunya dengan perbaikan kebijakan seleksi masuk Satuan \nPendidikan Negeri melalui dikeluarkannya  Petunjuk Teknis tentang \nPelaksanaan Sistem Penerimaan Murid Baru (SPMB) pada Satuan \nPendidikan Sekolah Menengah Atas  (SMA), Satuan Pendidikan Sekolah \nMenengah Kejuruan (SMK), dan Satuan P

In [7]:
character_split_texts = text_splitter.split_text('\n\n'.join(pdf_texts))

print(len(character_split_texts))
print(character_split_texts[0])

144
Juknis SPMB Jatim Tahun Ajaran 2025/2026  i 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Nomor : 100.3.6/1425/101.7.1/2025


In [9]:
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []

for text in character_split_texts:
    token_split_texts.extend(token_splitter.split_text(text))

print(len(token_split_texts))

print(token_split_texts[0])

392
juknis spmb jatim tahun ajaran 2025 / 2026 i nomor : 100. 3. 6 / 1425 / 101. 7. 1 / 2025


In [12]:


embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"},
)


def generate_embeddings(texts: list[str]) -> list[list[float]]:
    return embeddings.embed_documents(texts)

In [15]:
chunked_document_embeddings = generate_embeddings(token_split_texts)

print(len(chunked_document_embeddings))
print(chunked_document_embeddings[0])

392
[-0.09079959988594055, 0.12574724853038788, -0.008009877055883408, -0.07006149739027023, -0.04087882861495018, -0.028382068499922752, 0.03727249428629875, 0.04723357781767845, 0.014528957195580006, -0.021456167101860046, 0.0839720293879509, -0.053090132772922516, 0.04172040894627571, -0.008706442080438137, -0.027324948459863663, -0.03017139993607998, -0.043678976595401764, -0.0011491822078824043, 0.0015183743089437485, -0.046828947961330414, 0.027937093749642372, 0.03437500447034836, -0.012699904851615429, -0.03167238458991051, -0.0043570781126618385, -0.01513975765556097, -0.002862789435312152, 0.04114798083901405, 0.018916599452495575, -0.05674092099070549, -0.02869780734181404, 0.06598268449306488, -0.009862544015049934, -0.043153952807188034, -0.051921743899583817, -0.03667023032903671, -0.0425383485853672, 0.014284453354775906, 0.10247563570737839, 0.002715509617701173, 0.011779830791056156, -0.02101120911538601, -0.0270652174949646, -0.009864726103842258, 0.030634675174951553

In [16]:
def combine_vector_and_text(
        texts: list[str], embeddings: list[list[float]]
) -> list[dict[str, str]]:
    return [
        {
            "id": str(i),
            "values": embedding,
            "metadata": {
                "text": text,
                "chunk_size": len(text),
                "chunk_overlap": 0,
            },
        }
        for i, (embedding, text) in enumerate(zip(embeddings, texts))
    ]



In [17]:
data_with_embeddings = combine_vector_and_text(token_split_texts, chunked_document_embeddings)

print(len(data_with_embeddings))
print(data_with_embeddings[0])

392
{'id': '0', 'values': [-0.09079959988594055, 0.12574724853038788, -0.008009877055883408, -0.07006149739027023, -0.04087882861495018, -0.028382068499922752, 0.03727249428629875, 0.04723357781767845, 0.014528957195580006, -0.021456167101860046, 0.0839720293879509, -0.053090132772922516, 0.04172040894627571, -0.008706442080438137, -0.027324948459863663, -0.03017139993607998, -0.043678976595401764, -0.0011491822078824043, 0.0015183743089437485, -0.046828947961330414, 0.027937093749642372, 0.03437500447034836, -0.012699904851615429, -0.03167238458991051, -0.0043570781126618385, -0.01513975765556097, -0.002862789435312152, 0.04114798083901405, 0.018916599452495575, -0.05674092099070549, -0.02869780734181404, 0.06598268449306488, -0.009862544015049934, -0.043153952807188034, -0.051921743899583817, -0.03667023032903671, -0.0425383485853672, 0.014284453354775906, 0.10247563570737839, 0.002715509617701173, 0.011779830791056156, -0.02101120911538601, -0.0270652174949646, -0.009864726103842258

# Upload to pinecone

In [21]:
from pinecone import ServerlessSpec

pc.create_index(
    name="rag-pinecone",
    dimension=384,  # Replace with your model dimensions
    metric="cosine",  # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "rag-pinecone",
    "metric": "cosine",
    "host": "rag-pinecone-vnabbwx.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [22]:
index = pc.Index("rag-pinecone")

In [23]:
def upsert_data_to_pinecone(data_with_metadata: list[dict[str, any]]):
    index.upsert(vectors=data_with_metadata)


upsert_data_to_pinecone(data_with_embeddings)

In [25]:
def get_query_embedding(query: str) -> list[float]:
    return embeddings.embed_query(query)

In [33]:
query = "Berikan overview singkat apa itu spmb dan tahapan tahapan apa saja yang perlu untuk dipersiapkan"
query_embeddings = get_query_embedding(query)

In [34]:
def query_pinecone_index(
        query_embedding: list[float], top_k: int = 2, include_metadata: bool = True
) -> dict[str, any]:
    return index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=include_metadata,
    )


query_result = query_pinecone_index(query_embeddings, top_k=5)
print(query_result)

{'matches': [{'id': '281',
              'metadata': {'chunk_overlap': 0.0,
                           'chunk_size': 462.0,
                           'text': '##pada kepala satuan pendidikan sma / smk, '
                                   'orang tua / wali, calon murid baru, dan '
                                   'para pemangku kepentingan lainnya, agar '
                                   'pelaksanaan spmb dapat berjalan lancar, '
                                   'efektif, dan efisien, serta menyelesaikan '
                                   'permasalahan yang timbul berkaitan dengan '
                                   'pelaksanaan spmb sampai dengan melaporkan '
                                   'pelaksanaan spmb sma / smk sesuai dengan '
                                   'yang diharapkan. kepala dinas pendidikan '
                                   'provinsi jawa timur aries agung paewai '
                                   'pembina utama muda ( iv / c ) nip '
              

In [35]:
text_answer = " ".join([doc["metadata"]["text"] for doc in query_result["matches"]])

print(text_answer)

##pada kepala satuan pendidikan sma / smk, orang tua / wali, calon murid baru, dan para pemangku kepentingan lainnya, agar pelaksanaan spmb dapat berjalan lancar, efektif, dan efisien, serta menyelesaikan permasalahan yang timbul berkaitan dengan pelaksanaan spmb sampai dengan melaporkan pelaksanaan spmb sma / smk sesuai dengan yang diharapkan. kepala dinas pendidikan provinsi jawa timur aries agung paewai pembina utama muda ( iv / c ) nip 197604171995111001 juknis spmb jatim tahun ajaran 2025 / 2026 ii kepala dinas pendidikan provinsi jawa timur sesuai dengan undang - undang dasar negara republik indonesia tahun 1945 bahwa setiap warga negara berhak mendapat pendidikan yang bermutu dan berkeadilan. peningkatan dan pemerataan mutu pendidikan menjadi tantangan utama dalam pembangunan pendidikan di jawa timur. pemerintah jawa timur telah melakukan banyak hal untuk memperbaiki dan meningkatkan kualitas pendidikan yang bermutu dan berkeadilan, salah satunya dengan perbaikan kebijakan selek

In [36]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

In [37]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": (
                "Kamu adalah asisten yang ahli dan membantu dalam mencari informasi dari buku petunjuk teknis. "
                "Pengguna akan mengajukan pertanyaan berdasarkan informasi yang diambil dari buku petunjuk teknis. "
                "Jawablah pertanyaan pengguna hanya berdasarkan informasi yang diberikan."
            )
        },
        {
            "role": "user",
            "content": f"Pertanyaan: {query} \n\nInformasi: {retrieved_documents}"
        }
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content


In [39]:
rag_result = rag(query, text_answer)
print(rag_result)

SPMB adalah Sistem Penerimaan Murid Baru pada satuan pendidikan baik SMA maupun SMK. Tahapan-tahapan persiapan yang perlu dilakukan meliputi:
1. Menyusun pedoman pelaksanaan SPMB berdasarkan petunjuk teknis yang ditetapkan.
2. Memberikan pedoman kepada kepala SMA/SMK, orang tua/wali, calon murid baru, dan para pemangku kepentingan lainnya agar pelaksanaan SPMB dapat berjalan lancar, efektif, dan efisien.
3. Menyusun aturan persyaratan pendaftaran SPMB yang meliputi syarat usia, kewarganegaraan, dan lainnya sesuai dengan ketentuan yang berlaku.
